In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import sys
os.chdir('/home/samhuang/ML/')
sys.path.insert(0, '/home/samhuang/ML')
sys.path.insert(0, '/home/samhuang/ML/sample')
#sys.path.insert(0, '/home/samhuang/../public/Polar_new/samples')
from readTFR import *
from matplotlib import pyplot as plt
plt.switch_backend('agg')
import matplotlib as mpl
from sklearn.metrics import roc_curve, auc
import models
import math
import numpy as np
import tensorflow as tf
import json
import pandas as pd
import random
from tqdm import tqdm
from train_utils import *
import logging
from datetime import datetime, date

#physical_gpus = tf.config.list_physical_devices('GPU')
#print (physical_gpus)
#tf.config.experimental.set_memory_growth(physical_gpus[0], True)

In [5]:
#################  input and variables  #######################
# Data / training parameters.
train_epochs = 500
batch_size = 512
shuffle_size_tr = 1
patience = 20
min_delta = 0.
learning_rate = 1e-4
dim_image = [[75, 75], [[-0.8, 0.8], [-0.8, 0.8]]]
save_model_name = 'best_model_ternary_CNN_kappa0.15/'

# Input datasets
#data_folder = "sample/samples_kappa0.15/samples_kappa0.15/"
data_folder = "sample/samples_kappa0.15/VBF_H5pp_ww_jjjj_and_VBF_H5mm_ww_jjjj_and_VBF_H5z_zz_jjjj/"
#data_folder = "sample/samples_kappa0.15/VBF_H5pp_ww_jjjj_and_VBF_H5mm_ww_jjjj/"
#data_folder = "/home/samhuang/../public/Polar_new/samples/"
#data_folder = "samples/"
data_tr = data_folder+"train.tfrecord"
data_vl = data_folder+"valid.tfrecord" 
data_te = data_folder+"test.tfrecord" 


#######################  plotting setup  ##########
font = {'size'  : 14} 
fontsize=18
fontsize2=16
legendsize=16
plt.rc('font', **font)
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"]="in"
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.top'] = True
plt.rcParams["xtick.major.size"] = 6 
plt.rcParams['xtick.major.width'] = 1.5 
plt.rcParams["ytick.major.size"] = 6 
plt.rcParams['ytick.major.width'] = 1.5 

In [6]:
# Datasets
dataset_tr, tr_total = get_dataset(data_tr, repeat=False, 
                                   batch_size=batch_size, 
                                   dim_image=dim_image+[True], 
                                   shuffle=shuffle_size_tr)
dataset_vl, vl_total = get_dataset(data_vl, repeat=False, 
                                   batch_size=batch_size, 
                                   dim_image=dim_image+[True], 
                                   shuffle=0)

2022-09-07 16:33:15.293315: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-07 16:33:15.301798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2022-09-07 16:33:15.307111: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2022-09-07 16:33:15.307205: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-09-07 16:33:16.219335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Str

The data set contains images
The data set contains images


In [10]:
# Create the model  
model = models.CNN_ternary(dim_image=dim_image[0] + [2])
#print(model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), 
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=min_delta, verbose=1, patience=patience)
check_point    = tf.keras.callbacks.ModelCheckpoint(save_model_name, monitor='val_loss', verbose=1, save_best_only=True)


history = model.fit(dataset_tr, validation_data=dataset_vl , epochs=train_epochs, batch_size=batch_size, callbacks=[early_stopping, check_point], verbose=0)


Epoch 00001: val_loss improved from inf to 2.45640, saving model to best_model_ternary_CNN_kappa0.15/
INFO:tensorflow:Assets written to: best_model_ternary_CNN_kappa0.15/assets

Epoch 00002: val_loss improved from 2.45640 to 1.12564, saving model to best_model_ternary_CNN_kappa0.15/
INFO:tensorflow:Assets written to: best_model_ternary_CNN_kappa0.15/assets

Epoch 00003: val_loss improved from 1.12564 to 0.83357, saving model to best_model_ternary_CNN_kappa0.15/
INFO:tensorflow:Assets written to: best_model_ternary_CNN_kappa0.15/assets

Epoch 00004: val_loss improved from 0.83357 to 0.75707, saving model to best_model_ternary_CNN_kappa0.15/
INFO:tensorflow:Assets written to: best_model_ternary_CNN_kappa0.15/assets

Epoch 00005: val_loss improved from 0.75707 to 0.73383, saving model to best_model_ternary_CNN_kappa0.15/
INFO:tensorflow:Assets written to: best_model_ternary_CNN_kappa0.15/assets

Epoch 00006: val_loss improved from 0.73383 to 0.72721, saving model to best_model_ternary_CN

In [11]:
from print_and_draw import *
%load_ext autoreload
%autoreload 2
print_layer_and_params(model, history)


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Model: "CNN_ternary"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 512)               12127912  
_________________________________________________________________
dense_11 (Dense)             multiple                  1539      
Total params: 12,129,451
Trainable params: 12,129,447
Non-trainable params: 4
_________________________________________________________________
None

@LAYER1       @@@@@@@@@@@@@@@@@@@@@@
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 75, 75, 2)         0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 75, 75, 2)         8         


In [ ]:
print (history.history.keys())
print (history.params)
#model.save(save_model_name)

In [ ]:
%matplotlib inline
# Plot results curves.
fig = plt.figure(1, figsize=(10, 14))
fig.clf()
x = range(len(history.history['loss']))
# Loss.
y_tr = history.history['loss']
y_vl = history.history['val_loss']
ax = fig.add_subplot(2, 1, 1)
ax.plot(x, y_tr, "b", label="Training")
ax.plot(x, y_vl, "b--", label="Validation")
ax.set_title("Loss across training")
ax.set_xlabel("Training iteration")
ax.set_ylabel("Loss (categorical cross-entropy)")
ax.legend()
# Accuracy.
y_tr = history.history['accuracy']
y_vl = history.history['val_accuracy']
ax = fig.add_subplot(2, 1, 2)
ax.plot(x, y_tr, "b", label="Training")
ax.plot(x, y_vl, "b--", label="Test")
ax.set_title("Accuracy across training")
ax.set_xlabel("Training iteration")
ax.set_ylabel("Accuracy")
fig.show()

In [ ]:
loaded_model = tf.keras.models.load_model(save_model_name)

dataset_te, te_total  = get_dataset(data_te, repeat=False, 
                                    batch_size=1, 
                                    dim_image=dim_image+[True], 
                                    shuffle=0)

results = loaded_model.evaluate(dataset_te)
print("Testing Loss = {0:f}, Testing Accuracy = {1:f}".format(results[0], results[1]))

labels = [x[1][0].tolist() for x in dataset_te.as_numpy_iterator()]

dataset_te, te_total  = get_dataset(data_te, repeat=False, 
                                    batch_size=1, 
                                    dim_image=dim_image+[False], 
                                    shuffle=0)

predictions = loaded_model.predict(dataset_te).tolist()

data = {'test_scores': predictions, 'test_labels': labels}

print (predictions[:10])
print (labels[:10])





In [ ]:
confusion = confusion_matrix(labels, predictions)
print('Confusion Matrix:')
print(confusion)

In [ ]:
save_model_name = 'best_model_CNN_1/best'
loaded_model = tf.keras.models.load_model(save_model_name)

dim_image = [[75, 75], [[-0.8, 0.8], [-0.8, 0.8]]]
data_te = "sample/test.tfrecord"
dataset_te, te_total  = get_dataset(data_te, repeat=False, 
                                    batch_size=1, 
                                    dim_image=dim_image+[True], 
                                    shuffle=0)

results = loaded_model.evaluate(dataset_te)
print("Testing Loss = {0:f}, Testing Accuracy = {1:f}".format(results[0], results[1]))

labels = [x[1][0].tolist() for x in dataset_te.as_numpy_iterator()]

dataset_te, te_total  = get_dataset(data_te, repeat=False, 
                                    batch_size=1, 
                                    dim_image=dim_image+[False], 
                                    shuffle=0)

predictions = loaded_model.predict(dataset_te).tolist()

data = {'test_scores': predictions, 'test_labels': labels}

In [ ]:
%matplotlib inline
#predictions = loaded_model.predict(dataset_te)
#print ([x for x in dataset_te][0])
#labels = np.concatenate([y for x, y in dataset_te], axis=0)
labels = np.array(labels)
predictions = np.array(predictions)
n_class = np.shape(labels)[1]
print (n_class)

 
fpr, tpr, roc_auc = dict(), dict(), dict()
for i in range(n_class):
    fpr[i], tpr[i], _ = roc_curve(labels[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
 
plt.figure(figsize=(8,6))
signal=[r'$W^+$',r'$W^-$',r'$Z$']
for i in range(n_class):
    plt.plot(fpr[i], tpr[i], label='{0} (auc = {1:0.2f})'.format(signal[i], roc_auc[i]))
            
plt.plot([0, 1], [0, 1], color="navy", linestyle="--")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.title('ROC curve')
plt.legend(loc='lower right', fontsize=15)
plt.show()

In [ ]:
labels = data['test_labels']
scores = data['test_scores']

labels_EW_LL, scores_EW_LL = [], []
labels_EW_nonLL, scores_EW_nonLL  = [], []
labels_QCD, scores_QCD  = [], []

for idx in tqdm(range(len(labels))):
    if labels[idx][0] > 0.5 or labels[idx][1] > 0.5:
        labels_EW_LL.append([labels[idx][0], labels[idx][1]])
        total_EW_LL = scores[idx][0] + scores[idx][1]
        if int(scores[idx][2]) == 1:
            scores_EW_LL.append([0.5, 0.5])
        else:
            scores_EW_LL.append([scores[idx][0]/total_EW_LL, scores[idx][1]/total_EW_LL])
    if labels[idx][0] > 0.5 or labels[idx][2] > 0.5:
        labels_EW_nonLL.append([labels[idx][0], labels[idx][2]])
        total_EW_nonLL = scores[idx][0] + scores[idx][2]
        if int(scores[idx][1]) == 1:
            scores_EW_nonLL.append([0.5, 0.5])
        else:
            scores_EW_nonLL.append([scores[idx][0]/total_EW_nonLL, scores[idx][2]/total_EW_nonLL])
    if labels[idx][1] > 0.5 or labels[idx][2] > 0.5:
        labels_QCD.append([labels[idx][1], labels[idx][2]])
        total_QCD = scores[idx][1] + scores[idx][2]
        if int(scores[idx][0]) == 1:
            scores_QCD.append([0.5, 0.5])
        else:
            scores_QCD.append([scores[idx][1]/total_QCD, scores[idx][2]/total_QCD])

labels_EW_LL, scores_EW_LL = np.array(labels_EW_LL), np.array(scores_EW_LL)
labels_EW_nonLL, scores_EW_nonLL = np.array(labels_EW_nonLL), np.array(scores_EW_nonLL)
labels_QCD, scores_QCD = np.array(labels_QCD), np.array(scores_QCD)
        
from sklearn.metrics import roc_curve, auc

fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(2):
    fpr['EW_LL'], tpr['EW_LL'], _ = roc_curve(labels_EW_LL[:, i], scores_EW_LL[:, i])
    roc_auc['EW_LL'] = auc(fpr['EW_LL'], tpr['EW_LL'])
    
    fpr['EW_nonLL'], tpr['EW_nonLL'], _ = roc_curve(labels_EW_nonLL[:, i], scores_EW_nonLL[:, i])
    roc_auc['EW_nonLL'] = auc(fpr['EW_nonLL'], tpr['EW_nonLL'])
    
    fpr['QCD'], tpr['QCD'], _ = roc_curve(labels_QCD[:, i], scores_QCD[:, i])
    roc_auc['QCD'] = auc(fpr['QCD'], tpr['QCD'])

plt.figure(1, figsize=(8, 6))
lw = 2
plt.plot(
    fpr['EW_LL'],
    tpr['EW_LL'],
    color="red",
    lw=lw,
    label=r"ROC curve (area = %0.2f), EW, LL" % roc_auc['EW_LL'],
)
plt.plot(
    fpr['EW_nonLL'],
    tpr['EW_nonLL'],
    color="green",
    lw=lw,
    label=r"ROC curve (area = %0.2f), EW, non-LL" % roc_auc['EW_nonLL'],
)
plt.plot(
    fpr['QCD'],
    tpr['QCD'],
    color="blue",
    lw=lw,
    label=r"ROC curve (area = %0.2f), QCD" % roc_auc['QCD'],
)
plt.plot([0, 1], [0, 1], color="black", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Testing ROC Curve")
plt.legend(loc="lower right")
plt.show()